In [1]:
# !pip install scanorama

In [2]:
# !pip install scanorama
import numpy as np
import scanorama
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

scanpy==1.4+18.gaabe446 anndata==0.6.17 numpy==1.15.4 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
import utils_viz

In [4]:
# Create folder to save the results 
import os
dirname = os.getcwd()
print(dirname)
data_dir = os.path.join(dirname, './data/')
# if not os.path.exists('./results_cellatlas/'): os.makedirs('./results_cellatlas/')

if not os.path.exists('./results_cellatlas/scanorama_results/'): os.makedirs('./results_cellatlas/scanorama_results/')
    
save_dir = os.path.join(dirname, 'results_cellatlas/scanorama_results/')
def save_images(basename):
    if not os.path.exists('./figures_cellatlas/'): os.makedirs('./figures_cellatlas/')
    
    if not os.path.exists('./figures_cellatlas/scanorama_results/'): os.makedirs('./figures_cellatlas/scanorama_results/')
    outname = './figures_cellatlas/scanorama_results/' + basename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

/acrc/jinmiao/CJM_lab/hoatran/python_workspace/batch_effect_manuscript


In [5]:
# read data from read count text table, data from R: genes x cells
# scale data by number of UMI first then load scaled data
adata = sc.read_text(os.path.join(data_dir,'hvg_scaled_filtered_total_batch1_seqwell_batch2_10x_inverse.txt'),delimiter='\t',first_column_names=True,dtype='float64')
print(adata)  # 6954 x 15006
print(adata.obs_names[1:3])
print(adata.var_names[1:3])

sample_adata = pd.read_csv(os.path.join(data_dir,'filtered_total_sample_ext_organ_celltype_batch.txt'),header=0, index_col=0, sep='\t')
print(sample_adata.values.shape)
print(sample_adata.keys())

adata.obs['cell_type'] = sample_adata.loc[adata.obs_names,['ct']]
adata.obs['batch'] = sample_adata.loc[adata.obs_names,['batch']]
adata.obs['tissue'] = sample_adata.loc[adata.obs_names,['tissue']]
adata.obs['organ'] = sample_adata.loc[adata.obs_names,['organ']]
adata.obs['batchlb'] = sample_adata.loc[adata.obs_names,['batchlb']]
print(len(adata.obs['cell_type']))
print(len(adata.obs['batch']))
# Save output into h5ad, easy to access 
adata.write_h5ad(os.path.join(data_dir,'hvg_scaled_dataset2_cellatlas.h5ad'))

... storing 'cell_type' as categorical
... storing 'tissue' as categorical
... storing 'organ' as categorical
... storing 'batchlb' as categorical


AnnData object with n_obs × n_vars = 6954 × 15006 
Index(['T-cell_Thymus_2.CGAGTATAGTCGGTCGGT', 'T-cell_Thymus_1.TGAAGCTCAAAGCGCTTG'], dtype='object')
Index(['0610007N19Rik', '0610007P14Rik'], dtype='object')
(6954, 10)
Index(['nGene', 'nUMI', 'orig.ident', 'batch', 'ct', 'ct.orig', 'tissue',
       'organ', 'percent.mito', 'batchlb'],
      dtype='object')
6954
6954


In [9]:
print('Read filtered data')
# adata = sc.read_h5ad(os.path.join(data_dir,'dataset12_cellatlas.h5ad'))
import time
from datetime import timedelta

# Filtering data 
# sc.pp.filter_cells(adata, min_genes=300)
# sc.pp.filter_genes(adata, min_cells=10)
adata

Read filtered data


AnnData object with n_obs × n_vars = 6954 × 15006 
    obs: 'cell_type', 'batch', 'tissue', 'organ', 'batchlb'

In [10]:
from scanorama import correct, visualize, process_data
from scanorama import dimensionality_reduce

In [11]:
# df = pd.DataFrame(data=adata.X,
#                   index=adata.obs_names,
#                   columns=adata.var_names)

# Input of scanorama is a list of AnnData
# adata1 = adata[adata.obs['batch']=='Batch1',:].copy()
# print(adata1)
# adata2 = adata[adata.obs['batch']=='Batch2',:].copy()
# print(adata2)

adata1 = adata[adata.obs['batch']==1,:].copy()
print(adata1)
adata2 = adata[adata.obs['batch']==2,:].copy()
print(adata2)


Up to anndata 0.6.12, `.copy()` cast a non-'float32' `.X` to 'float32'. Now, the dtype 'float64' is maintained. 


AnnData object with n_obs × n_vars = 4239 × 15006 
    obs: 'cell_type', 'batch', 'tissue', 'organ', 'batchlb'


Up to anndata 0.6.12, `.copy()` cast a non-'float32' `.X` to 'float32'. Now, the dtype 'float64' is maintained. 


AnnData object with n_obs × n_vars = 2715 × 15006 
    obs: 'cell_type', 'batch', 'tissue', 'organ', 'batchlb'


In [12]:
adata_ls = [adata1, adata2]

In [13]:
# Batch correction.
# datasets = [df.values]
# List of datasets (matrices of cells-by-genes)
t1 = time.time()
corrected = scanorama.correct_scanpy(adata_ls,batch_size=50,return_dense=True, knn=20)
# corrected, genes = scanorama.correct(adata_ls, adata1.var_names,batch_size=30)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))

Found 15006 genes among all datasets
[[0.         0.60478821]
 [0.         0.        ]]
Processing datasets (0, 1)
Took 0:03:41.927960


In [14]:
print(corrected[0])
print(corrected[0].obs_names[0:10]==adata1.obs_names[0:10])

AnnData object with n_obs × n_vars = 4239 × 15006 
    obs: 'cell_type', 'batch', 'tissue', 'organ', 'batchlb'
[ True  True  True  True  True  True  True  True  True  True]


In [15]:
print(corrected[1])
print(corrected[1].obs_names[0:10]==adata2.obs_names[0:10])

AnnData object with n_obs × n_vars = 2715 × 15006 
    obs: 'cell_type', 'batch', 'tissue', 'organ', 'batchlb'
[ True  True  True  True  True  True  True  True  True  True]


In [16]:
total_ann = sc.AnnData(np.concatenate([corrected[0].X, corrected[1].X]))
print(total_ann)
# 

AnnData object with n_obs × n_vars = 6954 × 15006 


In [17]:
total_ann.var_names = corrected[0].var_names
total_ann.obs_names = corrected[0].obs_names.tolist() + corrected[1].obs_names.tolist()
total_ann.obs['cell_type'] = corrected[0].obs['cell_type'].tolist() + corrected[1].obs['cell_type'].tolist()
total_ann.obs['batch'] = corrected[0].obs['batch'].tolist() + corrected[1].obs['batch'].tolist()
total_ann.obs['organ'] = corrected[0].obs['organ'].tolist() + corrected[1].obs['organ'].tolist()
total_ann


AnnData object with n_obs × n_vars = 6954 × 15006 
    obs: 'cell_type', 'batch', 'organ'

In [18]:
sc.tl.pca(total_ann, svd_solver='arpack') # n_comps=20
print('before: ',total_ann.obsm['X_pca'][1:2])
total_ann.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
print('after multiply: ',total_ann.obsm['X_pca'][1:2])


before:  [[-1.5910061e-01  8.2543064e-03 -1.7969729e-01  3.0537101e-02
   2.3493743e-02  1.6812559e-03 -4.4159669e-02 -3.3346530e-02
  -1.8137686e-02 -3.9725851e-02  2.3152097e-03 -4.1205850e-03
   2.7589891e-02  6.4248610e-03  1.1727295e-02 -3.0916279e-05
  -6.0791505e-04 -4.9377825e-02  1.0796484e-03  4.3901369e-02
  -3.0145464e-03  2.6232991e-02 -2.3883017e-02 -4.6205181e-03
   7.0703961e-02 -7.5599842e-04 -4.7847543e-02  5.0840728e-02
  -2.0671509e-02  3.5236109e-02 -3.2553643e-02  9.6533056e-03
   2.9866224e-02 -3.6137532e-02 -1.2154238e-03 -9.0826415e-03
  -4.8011295e-03 -1.0851027e-02  1.6751131e-02  2.4429675e-02
  -2.1757390e-02  6.3180365e-03  9.6022226e-03  2.5035439e-02
  -4.4038682e-04  2.8487290e-03  3.5391316e-02 -3.7583537e-02
   6.5146405e-03  1.5780855e-02]]
after multiply:  [[ 1.5910061e-01 -8.2543064e-03  1.7969729e-01 -3.0537101e-02
  -2.3493743e-02 -1.6812559e-03  4.4159669e-02  3.3346530e-02
   1.8137686e-02  3.9725851e-02 -2.3152097e-03  4.1205850e-03
  -2.75898

In [30]:
# save_dir = os.path.join(dirname, 'results_cellatlas/scanorama_results/')
total_ann.write_h5ad(os.path.join(save_dir,'scanorama_corrected_dataset2.h5ad'))
# total_ann.write_csvs(save_dir, skip_data=False)

In [20]:
print(t1)
print(t2)
time_taken = t2 - t1
time_taken_mins = divmod(time_taken, 60)
time_taken_hours, rest = divmod( time_taken, 3600)
hours_mins, hours_secs = divmod( rest, 60)
print('Took seconds: '+str(timedelta(seconds=round(time_taken))))
print('Took minutes: '+str(time_taken_mins))
print('Took hours_minutes_seconds: ',str(time_taken_hours),str(hours_mins),str(hours_secs))
usecase_name = 'scanorama'
filename = 'scanorama_exetime.csv'

data = {'use_case':usecase_name, 'exetime_secs':str(round(time_taken)),
       'exetimehours': str(time_taken_hours),
       'exetimemins': str(hours_mins),
       'exetimesecs':str(round(hours_secs))} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['exetime'])
print(df)
print("Save file into directory: ",save_dir)
df.to_csv(save_dir+filename) 

1557803107.534985
1557803329.462945
Took seconds: 0:03:42
Took minutes: (3.0, 41.92795991897583)
Took hours_minutes_seconds:  0.0 3.0 41.92795991897583
          use_case exetime_secs exetimehours exetimemins exetimesecs
exetime  scanorama          222          0.0         3.0          42
Save file into directory:  /acrc/jinmiao/CJM_lab/hoatran/python_workspace/batch_effect_manuscript/results_cellatlas/scanorama_results/


In [21]:
# Function to plot TSNE
def plotTSNE(adata, color_group, n_pcs=20, perplexity=90, save_filename='tsne', use_repx = False):
    #adata.var_names_make_unique()
    if use_repx:
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity, use_rep='X')
    else:    
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity)
    sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
    save_images(save_filename)     
    
    
def plotUMAP(adata, color_group, save_filename, use_repx = False):
    
#     if use_repx:
#         sc.pp.neighbors(adata, use_rep='X')
#     else:    
#         sc.pp.neighbors(adata,n_neighbors=10, n_pcs=20)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename) 
    
   

In [22]:
# Visulization of normalized data
# sc.tl.pca(total_ann, svd_solver='arpack')
sc.pp.neighbors(total_ann,n_neighbors=15, n_pcs=20)
print('scanorama - batch effect correction')
color_group = ["batch","cell_type","organ"]
plotUMAP(total_ann, color_group, 'umap_scanorama_corrected')
plotTSNE(total_ann, color_group, 20, 90, 'tsne_scanorama_corrected')

computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:09.79) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
scanorama - batch effect correction
computing UMAP
    finished (0:00:25.08) --> added
    'X_umap', UMAP coordinates (adata.obsm)
computing tSNE
    using 'X_pca' with n_pcs = 20
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished (0:00:58.74) --> added
    'X_tsne', tSNE coordinates (adata.obsm)


In [23]:
# scanorama_ann.obsm['X_umap'].__class__
colnu = []
for i in range(total_ann.obsm['X_umap'].shape[1]):
    colnu.append("UMAP"+str(i+1))
df = pd.DataFrame(total_ann.obsm['X_umap'], columns=colnu, index=total_ann.obs_names)
df['batch'] = pd.Series(total_ann.obs['batch'], index=total_ann.obs_names)
df['celltype'] = pd.Series(total_ann.obs['cell_type'], index=total_ann.obs_names)
df['organ'] = pd.Series(total_ann.obs['organ'], index=total_ann.obs_names)
df.to_csv(save_dir+'scanorama_umap.csv')  

In [24]:
colnt = []
for i in range(total_ann.obsm['X_tsne'].shape[1]):
    colnt.append("tSNE_"+str(i+1))

df = pd.DataFrame(total_ann.obsm['X_tsne'], columns=colnt, index=total_ann.obs_names)
df['batch'] = pd.Series(total_ann.obs['batch'], index=total_ann.obs_names)
df['celltype'] = pd.Series(total_ann.obs['cell_type'], index=total_ann.obs_names)
df['organ'] = pd.Series(total_ann.obs['organ'], index=total_ann.obs_names)
df.to_csv(save_dir+'scanorama_tsne.csv')  

In [25]:
# sdir_name = os.path.join(dirname, 'results/scanorama_results/')
# scanorama_ann = sc.read_h5ad(os.path.join(sdir_name,'scanorama_dataset1_uc3.h5ad'))
total_ann.obsm['X_pca'].shape
print(total_ann.obsm['X_pca'][1:5])
def write_to_csv(mat, genesname, cellsname, filename, save_dir):
    if isinstance(mat, np.ndarray):
        df = pd.DataFrame(mat, columns=genesname, index=cellsname)
    else:
        df = pd.DataFrame(mat.toarray(), columns=genesname, index=cellsname)        
    
    df.to_csv(save_dir+filename)  
    
filename = 'scanorama_pca.csv'
coln_pca = []
for i in range(total_ann.obsm['X_pca'].shape[1]):
    coln_pca.append("X_pca"+str(i+1))
    

write_to_csv(total_ann.obsm['X_pca'], coln_pca, total_ann.obs_names,filename, save_dir)

[[ 1.59100607e-01 -8.25430639e-03  1.79697290e-01 -3.05371005e-02
  -2.34937426e-02 -1.68125588e-03  4.41596694e-02  3.33465301e-02
   1.81376860e-02  3.97258513e-02 -2.31520971e-03  4.12058504e-03
  -2.75898911e-02 -6.42486103e-03 -1.17272949e-02  3.09162788e-05
   6.07915048e-04  4.93778251e-02 -1.07964838e-03 -4.39013690e-02
   3.01454635e-03 -2.62329914e-02  2.38830168e-02  4.62051807e-03
  -7.07039610e-02  7.55998422e-04  4.78475429e-02 -5.08407280e-02
   2.06715092e-02 -3.52361090e-02  3.25536430e-02 -9.65330563e-03
  -2.98662242e-02  3.61375324e-02  1.21542381e-03  9.08264145e-03
   4.80112946e-03  1.08510274e-02 -1.67511310e-02 -2.44296752e-02
   2.17573904e-02 -6.31803647e-03 -9.60222259e-03 -2.50354391e-02
   4.40386822e-04 -2.84872903e-03 -3.53913158e-02  3.75835374e-02
  -6.51464052e-03 -1.57808550e-02]
 [ 2.63991624e-01 -5.29901721e-02  2.11852551e-01 -1.65866226e-01
   4.98137884e-02 -1.81593508e-01  4.24777865e-02  2.63327677e-02
   5.06838076e-02  6.08012155e-02 -3.5855

In [26]:
# Compute the silhouette coefficient score, compare the batch mixing level of filtered data and normalized data
from sklearn.metrics import silhouette_score
import seaborn as sns
sc.tl.pca(adata, svd_solver='arpack') #n_comps=20
adata.obsm['X_pca'] *= -1 # multiply by -1 to match Seurat, same scale as total_ann
sil_orig = silhouette_score(adata.obsm.X_pca[:, :20], adata.obs['batch'])
print(np.median(sil_orig))
# sil_true = silhouette_score(corrected_adata.obsm.X_pca[:, :2], adata_true.obs.Group)
sil_corr = silhouette_score(total_ann.obsm.X_pca[:, :20], total_ann.obs['batch'])
print(np.median(sil_corr))
sns.barplot(x=['Filtered', 'Normalized'], y=[sil_orig, sil_corr])
pl.title('Silhouette Coeff Batch Scanorama')
save_images('scanorama_silhouette_coeff_batch_label')

# # Creates pandas DataFrame. 
usecase_name = 'scanorama_asw'
data_sil = {'use_case':usecase_name, 'sil_filtered':sil_orig,
       'sil_corrected': sil_corr}
df_score = pd.DataFrame(data_sil, index =['silscore'])
print(df_score)
print('Save data into directory: ',save_dir)
df_score.to_csv(save_dir+usecase_name+'.csv') 

0.115240835
0.05621321
               use_case  sil_filtered  sil_corrected
silscore  scanorama_asw      0.115241       0.056213
Save data into directory:  /acrc/jinmiao/CJM_lab/hoatran/python_workspace/batch_effect_manuscript/results_cellatlas/scanorama_results/


In [27]:
# Check the changement of values before scGen and after batch correction using scGen 
import scipy.stats as st
def describe_data(adata, ax, indx):    
    desc = st.describe(adata.X[indx], axis=ax)
    print('# of observations:', desc.nobs)
    print('min: %d\nmax: %d' % desc.minmax)
    print('mean: %.1f' % desc.mean)
    
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 10 in ",adata.obs['batch'][10],adata.obs_names[10])
print("Gene expression values before scanorama:")
# describe_data(adata, 0, 10) #cell id 10 in batch 1
print(adata.X[10])
print("Cell id: 10 in ",total_ann.obs['batch'][10],total_ann.obs_names[10])
print("Gene expression values after scanorama:")
print(total_ann.X[10])
# describe_data(total_ann, 0, 10) #cell id 10 in batch 1

Cell id: 10 in  1 T-cell_Thymus_2.GGTACAATGGCGTTTAGG
Gene expression values before scanorama:
[10.         -0.03701661 -0.2755073  ... -0.47979982 -0.1505914
 -0.15106844]
Cell id: 10 in  1 T-cell_Thymus_2.GGTACAATGGCGTTTAGG
Gene expression values after scanorama:
[ 0.10049654 -0.000372   -0.00276875 ... -0.00151819 -0.00073508
 -0.00089895]


In [28]:
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 500 in ",adata.obs['batch'][500],adata.obs_names[500])
print("Gene expression values before scanorama:")
describe_data(adata, 0, 500) #cell id 10 in batch 1
print(adata.X[500])
print("Cell id: 500 in ",total_ann.obs['batch'][500],total_ann.obs_names[500])
print("Gene expression values after scanorama:")
print(total_ann.X[500])
describe_data(total_ann, 0, 500) #cell id 10 in batch 

Cell id: 500 in  1 T-cell_Spleen_1.GTGGTACCATCTTAGAGA
Gene expression values before scanorama:
# of observations: 15006
min: -1
max: 10
mean: -0.0
[-0.038193   -0.03682071 -0.27553237 ... -0.47984665 -0.15050851
 -0.15106755]
Cell id: 500 in  1 T-cell_Spleen_1.GTGGTACCATCTTAGAGA
Gene expression values after scanorama:
[-0.00039638 -0.00038214 -0.0028596  ... -0.00156785 -0.0007596
 -0.00092546]
# of observations: 15006
min: 0
max: 0
mean: -0.0


In [29]:
adata

AnnData object with n_obs × n_vars = 6954 × 15006 
    obs: 'cell_type', 'batch', 'tissue', 'organ', 'batchlb'
    uns: 'pca'
    obsm: 'X_pca'
    varm: 'PCs'